In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('../..')

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from scipy import mean
from scipy import stats

from src.utils.aggregation import filter_data, aggregate_methods
from src.compare_methods import run_CD, run_dolan_more, bayes_scores

%load_ext autoreload
%autoreload 2

In [3]:
agg_metrics = pd.read_csv("notebooks/results_aggregation/agg_metrics.csv", index_col=0)
ml_100k = pd.read_csv("notebooks/results_aggregation/metrics.csv", index_col=0)

In [4]:
agg_metrics

,Method,Dataset,Metric,k,Value
0,most_popular,mts_library,precision,5,0.002639
1,most_popular,mts_library,precision,10,0.007337
2,most_popular,mts_library,precision,20,0.008621
3,most_popular,mts_library,precision,100,0.016131
4,most_popular,mts_library,recall,5,0.002495
...,...,...,...,...,...
9971,implicit_als,movielens_10m,time_fit,100,2.746746
9972,implicit_als,movielens_10m,time_predict,5,2.031604
9973,implicit_als,movielens_10m,time_predict,10,2.031604
9974,implicit_als,movielens_10m,time_predict,20,2.031604


In [5]:
filtered_metrics = filter_data(agg_metrics, "ndcg", 10, drop_minor_datasets=True)

In [6]:
filtered_metrics = pd.concat((filtered_metrics, ml_100k), ignore_index=True)

In [7]:
filtered_metrics_5 = filtered_metrics[filtered_metrics["Dataset"].isin(
    ("epinions", "amazon_finefoods", "movielens_20m", "movielens_100k", "goodreads")
)].copy()

In [23]:
filtered_metrics_9 = filtered_metrics[filtered_metrics["Dataset"].isin(
    (
        "epinions",
        "amazon_finefoods",
        "movielens_20m",
        "movielens_100k",
        "goodreads",
        "yelp",
        "movielens_1m",
        "gowalla",
        "movielens_10m"
    )
)].copy()

In [8]:
results = run_CD(filtered_metrics_5, save_image=True, image_path=Path('.'))

Pairwise testing:   0%|          | 0/45 [00:00<?, ?it/s]

In [10]:
def calculate_metrics(data, votenrank_list = None, drop_cd: bool = False,) -> pd.DataFrame:

    results_DM = run_dolan_more(data, image_path=Path('.'))
    results_DM.rename(columns={"score": "dm_score", "ranks": "dm_value"}, inplace=True)
    if not drop_cd:
        results_CD = run_CD(data)
        results_CD.rename(columns={"score": "cd_score", "ranks": "cd_ranks"}, inplace=True)
        q = pd.merge(results_DM, results_CD, on=['Model_name'])
    else:
        q = results_DM
    # print(q)
    # results_new_DM = run_dolan_more(data, image_path=Path('.'), mode="leave_best_out")
    #results_new_DM.rename(columns={"score": "dm_lbo_score", "ranks": "dm_lbo_value"}, inplace=True)
    # q['dm_lbo_value'] = q['Model_name'].map(results_new_DM)

    #q = pd.merge(q, results_new_DM, on=['Model_name'])
    

    q.set_index("Model_name", inplace=True)
    metrics_MR = data.copy()



    metrics_MR['gm_score'] = metrics_MR.groupby("Method")['Value'].transform(stats.gmean)
    metrics_MR['mr_score'] = metrics_MR.groupby("Method")['Value'].transform(mean)
    metrics_MR['hm_score'] = metrics_MR.groupby("Method")['Value'].transform(stats.hmean)
    
    metrics_MR = metrics_MR.drop(columns=['Dataset', 'Value']).drop_duplicates()
    
    metrics_MR['gm_value'] = metrics_MR['gm_score'].rank(method='average', ascending=False)
    metrics_MR['mr_value'] = metrics_MR['mr_score'].rank(method='average', ascending=False)
    metrics_MR['hm_value'] = metrics_MR['hm_score'].rank(method='average', ascending=False)
    metrics_MR.set_index('Method', inplace=True)
    q = pd.merge(q, metrics_MR, right_index=True, left_index=True)

    # if votenrank_list is not None:
    #     metrics_votenrank = run_printtable_votenrank(data)
    #     for x in votenrank_list:
    #         metrics_method = metrics_votenrank[x].reset_index().rename(columns={'Ranking position': f'{x}_value'})
    #         metrics_method['Model_name'] = metrics_method.apply(lambda y: str(y[x]).split(': ')[1].split('\n')[0], axis=1)
    #         metrics_method[x + '_score'] = metrics_method.apply(lambda y: float(str(y[x]).split(': ')[0]), axis=1)
    #         metrics_method.set_index("Model_name", inplace=True)
    #         #metrics_method.drop(columns=[x], inplace=True)
    #         q = pd.merge(q, metrics_method, right_index=True, left_index=True)
    return q

In [19]:
z_5 = calculate_metrics(filtered_metrics_5)

Pairwise testing:   0%|          | 0/45 [00:00<?, ?it/s]

In [24]:
z_9 = calculate_metrics(filtered_metrics_9)

Pairwise testing:   0%|          | 0/45 [00:00<?, ?it/s]

In [20]:
z_full = calculate_metrics(filtered_metrics)

Pairwise testing:   0%|          | 0/45 [00:00<?, ?it/s]

In [25]:
z_9

,dm_score,dm_value,cd_score,cd_ranks,gm_score,mr_score,hm_score,gm_value,mr_value,hm_value
ALS,0.077821,7.0,5.666667,6.0,0.018630,0.050393,0.006050,8.0,8.0,7.0
BPR,0.047665,9.0,6.888889,8.0,0.015326,0.046761,0.005662,9.0,9.0,8.0
EASE,0.161479,1.0,2.555556,1.0,0.056608,0.084179,0.036387,1.0,1.0,1.0
ItemKNN,0.143969,4.0,3.666667,3.5,0.048206,0.074669,0.030536,4.0,4.0,3.0
LightFM,0.148833,3.0,3.666667,3.5,0.050137,0.078974,0.029364,3.0,2.0,4.0
MostPop,0.070525,8.0,7.111111,9.0,0.021256,0.054430,0.008745,6.0,7.0,6.0
MultiVAE,0.151751,2.0,3.333333,2.0,0.051408,0.077074,0.032008,2.0,3.0,2.0
SASRec,0.099222,5.0,6.666667,7.0,0.030359,0.062612,0.012163,5.0,6.0,5.0
SLIM,0.098735,6.0,5.555556,5.0,0.018773,0.069696,0.000409,7.0,5.0,9.0
random,0.000000,10.0,9.888889,10.0,0.000693,0.004177,0.000202,10.0,10.0,10.0


In [22]:
z_full

,dm_score,dm_value,cd_score,cd_ranks,gm_score,mr_score,hm_score,gm_value,mr_value,hm_value
ALS,0.077975,8.0,5.800000,6.0,0.011085,0.035555,0.000677,8.0,8.0,8.0
BPR,0.059645,9.0,7.066667,9.0,0.009208,0.031362,0.000450,9.0,9.0,9.0
EASE,0.156386,1.0,2.433333,1.0,0.044626,0.068610,0.027871,1.0,1.0,1.0
ItemKNN,0.130492,4.0,4.333333,5.0,0.035028,0.057456,0.018819,4.0,5.0,4.0
LightFM,0.135147,3.0,3.833333,2.0,0.037370,0.060216,0.022021,3.0,2.0,2.0
MostPop,0.083503,7.0,6.766667,8.0,0.019805,0.042014,0.007386,7.0,7.0,6.0
MultiVAE,0.138493,2.0,4.233333,3.0,0.037726,0.059732,0.021691,2.0,3.0,3.0
SASRec,0.100815,6.0,6.433333,7.0,0.026121,0.056608,0.010329,5.0,6.0,5.0
SLIM,0.117544,5.0,4.266667,4.0,0.022788,0.058680,0.000948,6.0,4.0,7.0
random,0.000000,10.0,9.833333,10.0,0.000690,0.003240,0.000182,10.0,10.0,10.0


In [28]:
spearman_5_ma = stats.spearmanr(z_5["cd_ranks"], z_full["cd_ranks"])[0]
spearman_9_ma = stats.spearmanr(z_9["cd_ranks"], z_full["cd_ranks"])[0]
spearman_5_gm = stats.spearmanr(z_5["gm_value"], z_full["gm_value"])[0]
spearman_9_gm = stats.spearmanr(z_9["gm_value"], z_full["gm_value"])[0]

In [29]:
spearman_5_ma, spearman_9_ma, spearman_5_gm, spearman_9_gm

(0.8875420937728943,
 0.9483326481409007,
 0.9393939393939393,
 0.9878787878787878)